## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [5]:
lng = 'fr'
acteur = 'ciusss_centresud'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [6]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [7]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Address', 'Title', 'Type', 'text']]
    return df

In [8]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 854 documents.


In [9]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Corpus,Sous-corpus,Address,Title,Type,text
0,ciusss_centresud,Aides techniques / Prothèses et orthèses,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,"Aides à la marche (cannes, béquilles, marchettes)",text//html,"Déficience physique (sensorielle, langage, aud..."
1,ciusss_centresud,Aides techniques / Prothèses et orthèses,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,Aides de suppléance à l’audition,text//html,"Déficience physique (sensorielle, langage, aud..."
2,ciusss_centresud,Aides techniques / Prothèses et orthèses,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,Ambulateurs,text//html,"Déficience physique (sensorielle, langage, aud..."
3,ciusss_centresud,Aides techniques / Prothèses et orthèses,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,"Chaussures orthétiques, orthèses plantaires et...",text//html,"Déficience physique (sensorielle, langage, aud..."
4,ciusss_centresud,Aides techniques / Prothèses et orthèses,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,Chien d’assistance à la motricité,text//html,"Déficience physique (sensorielle, langage, aud..."
...,...,...,...,...,...,...
437,ciusss_centresud,Soins et services,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,courslsq.ccsmtl@ssss.gouv.qc.ca,NaN,"Page introuvable Désolé, il est impossible de ..."
438,ciusss_centresud,Soins et services,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,"4689, avenue Papineau",NaN,"Page introuvable Désolé, il est impossible de ..."
439,ciusss_centresud,Soins et services,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,physio.hv.ccsmtl@ssss.gouv.qc.ca,NaN,"Page introuvable Désolé, il est impossible de ..."
440,ciusss_centresud,Soins et services,https:////ciusss-centresudmtl.gouv.qc.ca//soin...,COVID-19 : Consultez la section Consultation a...,NaN,"Page introuvable Désolé, il est impossible de ..."


### **Nettoyage**

In [10]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [11]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 854 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [12]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [13]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [14]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 330374 tokens.
Le POS tagging devrait prendre environ 0 minutes.


In [15]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [16]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [17]:
#tagged

### **Lemmatisation** (FrenchLefffLemmatizer)
Sur pause ; peut-être qu'il vaudrait mieux le faire plus tard (au moment d'identifier si chaque terme existe déjà dans la taxo ou non) afin d'éviter de complexifier toutes les autres étapes

In [18]:
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

#### **Mapping POS Tags** (FRMG)
On doit mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)
#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [19]:
# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [20]:
# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

In [21]:
# tagged_dict = df_tagged.to_dict('records')

In [22]:
# # tokens = df_tagged['Token'].tolist()
# # tags = df_tagged['Tag Lefff'].tolist()
# # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

# for term in tagged_dict:
#     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
#         term_l = lemmatizer.lemmatize(word=term['Token'])
    
#     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
#         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

#     else:
#         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

#     term['Lemme'] = term_l

In [23]:
# tagged_dict

In [24]:
# tagged = DataFrame(tagged_dict)
# tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()

# tagged

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [25]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=10))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [26]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 3384955 ngrammes.


In [27]:
ngrammes

[(['consultation', 'NOM'],),
 (['consultation', 'NOM'], ['avec', 'PRP']),
 (['consultation', 'NOM'], ['avec', 'PRP'], ['un', 'DET:ART']),
 (['consultation', 'NOM'],
  ['avec', 'PRP'],
  ['un', 'DET:ART'],
  ['médecin', 'NOM']),
 (['consultation', 'NOM'],
  ['avec', 'PRP'],
  ['un', 'DET:ART'],
  ['médecin', 'NOM'],
  ['STOP', 'INT']),
 (['consultation', 'NOM'],
  ['avec', 'PRP'],
  ['un', 'DET:ART'],
  ['médecin', 'NOM'],
  ['STOP', 'INT'],
  ['une', 'DET:ART']),
 (['consultation', 'NOM'],
  ['avec', 'PRP'],
  ['un', 'DET:ART'],
  ['médecin', 'NOM'],
  ['STOP', 'INT'],
  ['une', 'DET:ART'],
  ['infirmière', 'NOM']),
 (['consultation', 'NOM'],
  ['avec', 'PRP'],
  ['un', 'DET:ART'],
  ['médecin', 'NOM'],
  ['STOP', 'INT'],
  ['une', 'DET:ART'],
  ['infirmière', 'NOM'],
  ['ou', 'KON']),
 (['consultation', 'NOM'],
  ['avec', 'PRP'],
  ['un', 'DET:ART'],
  ['médecin', 'NOM'],
  ['STOP', 'INT'],
  ['une', 'DET:ART'],
  ['infirmière', 'NOM'],
  ['ou', 'KON'],
  ['un', 'DET:ART']),
 (['consu

### **Extraction des patrons syntaxiques**

In [28]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [29]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [30]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [31]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [32]:
frequencies

FreqDist({('de',): 17926, ('STOP',): 17826, ('et',): 7688, ('.',): 7678, ('la',): 7404, ("d'",): 6356, ('à',): 6330, ('vous',): 5706, ('les',): 5494, ('des',): 5470, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [33]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [34]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [35]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [36]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [37]:
phrases = filter_len(phrases)

In [38]:
phrases

[[('consultation',), ['NOM']],
 [('consultation', 'avec', 'un', 'médecin'), ['NOM', 'PRP', 'DET:ART', 'NOM']],
 [('médecin',), ['NOM']],
 [('infirmière',), ['NOM']],
 [('infirmière', 'ou', 'un', 'travailleur'), ['NOM', 'KON', 'DET:ART', 'ADJ']],
 [('infirmière', 'ou', 'un', 'travailleur', 'social'),
  ['NOM', 'KON', 'DET:ART', 'ADJ', 'ADJ']],
 [('infirmière', 'ou', 'un', 'travailleur', 'social', 'néphrologie'),
  ['NOM', 'KON', 'DET:ART', 'ADJ', 'ADJ', 'NOM']],
 [('infirmière',
   'ou',
   'un',
   'travailleur',
   'social',
   'néphrologie',
   'la',
   'clinique'),
  ['NOM', 'KON', 'DET:ART', 'ADJ', 'ADJ', 'NOM', 'DET:ART', 'NOM']],
 [('infirmière',
   'ou',
   'un',
   'travailleur',
   'social',
   'néphrologie',
   'la',
   'clinique',
   'de',
   'néphrologie'),
  ['NOM',
   'KON',
   'DET:ART',
   'ADJ',
   'ADJ',
   'NOM',
   'DET:ART',
   'NOM',
   'PRP',
   'NOM']],
 [('néphrologie',), ['NOM']],
 [('néphrologie', 'la', 'clinique'), ['NOM', 'DET:ART', 'NOM']],
 [('néphrologie

On retire les n-grammes qui apparaissent moins de 15 fois dans le corpus

In [39]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 15]

In [40]:
phrases = filter_freq(phrases)

In [41]:
phrases

[[('consultation',), ['NOM']],
 [('consultation', 'avec', 'un', 'médecin'), ['NOM', 'PRP', 'DET:ART', 'NOM']],
 [('médecin',), ['NOM']],
 [('infirmière',), ['NOM']],
 [('infirmière', 'ou', 'un', 'travailleur'), ['NOM', 'KON', 'DET:ART', 'ADJ']],
 [('infirmière', 'ou', 'un', 'travailleur', 'social'),
  ['NOM', 'KON', 'DET:ART', 'ADJ', 'ADJ']],
 [('néphrologie',), ['NOM']],
 [('clinique',), ['NOM']],
 [('néphrologie',), ['NOM']],
 [('services',), ['NOM']],
 [('services', 'pour', 'traiter'), ['NOM', 'PRP', 'VER:infi']],
 [('services', 'pour', 'traiter', 'les', 'troubles'),
  ['NOM', 'PRP', 'VER:infi', 'DET:ART', 'NOM']],
 [('services', 'pour', 'traiter', 'les', 'troubles', 'et', 'maladies'),
  ['NOM', 'PRP', 'VER:infi', 'DET:ART', 'NOM', 'KON', 'NOM']],
 [('troubles',), ['NOM']],
 [('troubles', 'et', 'maladies'), ['NOM', 'KON', 'NOM']],
 [('maladies',), ['NOM']],
 [('services',), ['NOM']],
 [('médecin',), ['NOM']],
 [('médecin', 'doit', 'en', 'faire'), ['NOM', 'VER:pres', 'PRP', 'VER:infi

In [42]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [43]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [44]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 145290 occurrences de ngrammes.


In [45]:
phrases[:15]

[[('consultation',), 'NOM', 492],
 [('consultation', 'avec', 'un', 'médecin'), 'NOM PRP DET:ART NOM', 280],
 [('médecin',), 'NOM', 1590],
 [('infirmière',), 'NOM', 490],
 [('infirmière', 'ou', 'un', 'travailleur'), 'NOM KON DET:ART ADJ', 240],
 [('infirmière', 'ou', 'un', 'travailleur', 'social'),
  'NOM KON DET:ART ADJ ADJ',
  240],
 [('néphrologie',), 'NOM', 20],
 [('clinique',), 'NOM', 740],
 [('néphrologie',), 'NOM', 20],
 [('services',), 'NOM', 3252],
 [('services', 'pour', 'traiter'), 'NOM PRP VER:infi', 102],
 [('services', 'pour', 'traiter', 'les', 'troubles'),
  'NOM PRP VER:infi DET:ART NOM',
  60],
 [('services', 'pour', 'traiter', 'les', 'troubles', 'et', 'maladies'),
  'NOM PRP VER:infi DET:ART NOM KON NOM',
  36],
 [('troubles',), 'NOM', 166],
 [('troubles', 'et', 'maladies'), 'NOM KON NOM', 42]]

In [46]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('consultation',), 'NOM', 492],
 [('consultation', 'avec', 'un', 'médecin'), 'NOM PRP DET:ART NOM', 280],
 [('médecin',), 'NOM', 1590],
 [('infirmière',), 'NOM', 490],
 [('infirmière', 'ou', 'un', 'travailleur'), 'NOM KON DET:ART ADJ', 240],
 [('infirmière', 'ou', 'un', 'travailleur', 'social'),
  'NOM KON DET:ART ADJ ADJ',
  240],
 [('néphrologie',), 'NOM', 20],
 [('clinique',), 'NOM', 740],
 [('néphrologie',), 'NOM', 20],
 [('services',), 'NOM', 3252],
 [('services', 'pour', 'traiter'), 'NOM PRP VER:infi', 102],
 [('services', 'pour', 'traiter', 'les', 'troubles'),
  'NOM PRP VER:infi DET:ART NOM',
  60],
 [('services', 'pour', 'traiter', 'les', 'troubles', 'et', 'maladies'),
  'NOM PRP VER:infi DET:ART NOM KON NOM',
  36],
 [('troubles',), 'NOM', 166],
 [('troubles', 'et', 'maladies'), 'NOM KON NOM', 42],
 [('maladies',), 'NOM', 146],
 [('services',), 'NOM', 3252],
 [('médecin',), 'NOM', 1590],
 [('médecin', 'doit', 'en', 'faire'), 'NOM VER:pres PRP VER:infi', 210],
 [('médecin', 

In [47]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 2332 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [48]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # Pour prendre les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [49]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [50]:
terms = filter_patterns(phrases)
#terms = phrases

In [51]:
terms

[[('consultation',), 'NOM', 492],
 [('consultation', 'avec', 'un', 'médecin'), 'NOM PRP DET:ART NOM', 280],
 [('médecin',), 'NOM', 1590],
 [('infirmière',), 'NOM', 490],
 [('néphrologie',), 'NOM', 20],
 [('clinique',), 'NOM', 740],
 [('néphrologie',), 'NOM', 20],
 [('services',), 'NOM', 3252],
 [('services', 'pour', 'traiter'), 'NOM PRP VER:infi', 102],
 [('services', 'pour', 'traiter', 'les', 'troubles'),
  'NOM PRP VER:infi DET:ART NOM',
  60],
 [('troubles',), 'NOM', 166],
 [('troubles', 'et', 'maladies'), 'NOM KON NOM', 42],
 [('maladies',), 'NOM', 146],
 [('services',), 'NOM', 3252],
 [('médecin',), 'NOM', 1590],
 [('demande',), 'NOM', 766],
 [('accès',), 'NOM', 854],
 [('accès', 'aux', 'services'), 'NOM PRP:det NOM', 484],
 [('services',), 'NOM', 3252],
 [('hôpital',), 'NOM', 1724],
 [('hôpital', 'de', 'verdun'), 'NOM PRP NOM', 452],
 [('hôpital', 'de', 'verdun', 'pour', 'prendre'),
  'NOM PRP NOM PRP VER:infi',
  96],
 [('verdun',), 'NOM', 726],
 [('verdun', 'pour', 'prendre'), 

In [52]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 24 % des termes
On avait 145290 ngrammes, on en a maintenant 110936.


In [53]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [54]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [55]:
terms_patterns

[{'Expression': ('consultation',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 492},
 {'Expression': ('consultation', 'avec', 'un', 'médecin'),
  'Structure syntaxique': 'NOM PRP DET:ART NOM',
  'Fréquence': 280},
 {'Expression': ('médecin',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1590},
 {'Expression': ('infirmière',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 490},
 {'Expression': ('néphrologie',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 20},
 {'Expression': ('clinique',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 740},
 {'Expression': ('néphrologie',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 20},
 {'Expression': ('services',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 3252},
 {'Expression': ('services', 'pour', 'traiter'),
  'Structure syntaxique': 'NOM PRP VER:infi',
  'Fréquence': 102},
 {'Expression': ('services', 'pour', 'traiter', 'les', 'troubles'),
  'Structure syntaxique': 'NOM PRP VER:infi DET:ART NOM',
  'Fréquence': 60},
 {'

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [56]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [57]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 110936 ngrammes.


In [58]:
frequencies

FreqDist({('de',): 17926, ('STOP',): 17826, ('et',): 7688, ('.',): 7678, ('la',): 7404, ("d'",): 6356, ('à',): 6330, ('vous',): 5706, ('les',): 5494, ('des',): 5470, ...})

In [59]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [60]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.01 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [61]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [62]:
significant_coll


[{'Terme': 'consultation',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 492,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('consultation', 'avec', 'un', 'médecin'),
  'Structure syntaxique': 'NOM PRP DET:ART NOM',
  'Fréquence (TF)': 280,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': 'médecin',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1590,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'infirmière',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 490,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'néphrologie',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 20,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'clinique',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 740,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'néphrologie',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 20,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'services',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 3252,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('services', 'pour

In [63]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [64]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
67855,services,NOM,3252,-,-
69052,hôpital,NOM,1724,-,-
76535,médecin,NOM,1590,-,-
42103,centre,NOM,1348,-,-
74804,carte,NOM,902,-,-
...,...,...,...,...,...
12470,services alimentaires,NOM ADJ,16,123.05958,0.0
60974,loyer,NOM,16,-,-
8551,clicsanté,NOM,16,-,-
71071,accès clinique,NOM ADJ,16,40.987713,0.0


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [65]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [66]:
max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 98% des documents 
max_df

837

In [67]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

668

In [68]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [69]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > 0} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [70]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [71]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,services,622
1,hôpital,262
2,médecin,364
3,centre,448
4,carte,212
...,...,...
1615,services alimentaires,6
1616,loyer,6
1617,clicsanté,2
1618,accès clinique,4


In [72]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [73]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,services,NOM,3252,-,-,622
1,hôpital,NOM,1724,-,-,262
2,médecin,NOM,1590,-,-,364
3,centre,NOM,1348,-,-,448
4,carte,NOM,902,-,-,212
...,...,...,...,...,...,...
1530,itss et contraception,NOM KON NOM,16,286.332206,0.0,16
1529,équité salariale,NOM ADJ,16,288.022399,0.0,4
1528,consommation d'alcool soutien,NOM PRP NOM NOM,16,197.078397,0.0,16
1527,crlb et ird,NOM KON ADJ,16,294.375033,0.0,16


In [74]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [75]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [76]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [77]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [78]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [79]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [80]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [81]:
df['isMeSHTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,services,NOM,3252,-,-,622,False
1,hôpital,NOM,1724,-,-,262,False
2,médecin,NOM,1590,-,-,364,False
3,centre,NOM,1348,-,-,448,False
4,carte,NOM,902,-,-,212,False
...,...,...,...,...,...,...,...
1530,itss et contraception,NOM KON NOM,16,286.332206,0.0,16,False
1529,équité salariale,NOM ADJ,16,288.022399,0.0,4,False
1528,consommation d'alcool soutien,NOM PRP NOM NOM,16,197.078397,0.0,16,False
1527,crlb et ird,NOM KON ADJ,16,294.375033,0.0,16,False


In [82]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [83]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [84]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,services,NOM,3252,-,-,622,False
1,hôpital,NOM,1724,-,-,262,False
2,médecin,NOM,1590,-,-,364,False
3,centre,NOM,1348,-,-,448,False
4,carte,NOM,902,-,-,212,False
...,...,...,...,...,...,...,...
1530,itss et contraception,NOM KON NOM,16,286.332206,0.0,16,False
1529,équité salariale,NOM ADJ,16,288.022399,0.0,4,False
1528,consommation d'alcool soutien,NOM PRP NOM NOM,16,197.078397,0.0,16,False
1527,crlb et ird,NOM KON ADJ,16,294.375033,0.0,16,False


In [85]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,services,NOM,3252,-,-,622,False
1,hôpital,NOM,1724,-,-,262,False
2,médecin,NOM,1590,-,-,364,False
3,centre,NOM,1348,-,-,448,False
4,carte,NOM,902,-,-,212,False
...,...,...,...,...,...,...,...
1530,itss et contraception,NOM KON NOM,16,286.332206,0.0,16,False
1529,équité salariale,NOM ADJ,16,288.022399,0.0,4,True
1528,consommation d'alcool soutien,NOM PRP NOM NOM,16,197.078397,0.0,16,False
1527,crlb et ird,NOM KON ADJ,16,294.375033,0.0,16,False


In [86]:
# Test - ne retenir uniquement que les termes qui sont des MeSH ? 
df[df['isMeSHTerm'] == 'True']

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
54,travailleur social,NOM ADJ,320,-inf,1.0,274,True
57,assurance maladie,NOM NOM,318,-inf,1.0,228,True
289,douleur chronique,NOM ADJ,94,1321.546655,0.0,26,True
331,déficience intellectuelle,NOM ADJ,82,1012.991854,0.0,42,True
371,services sociaux,NOM ADJ,74,469.894963,0.0,48,True
387,soins infirmiers,NOM NOM,72,848.684707,0.0,40,True
465,santé mentale,NOM ADJ,60,732.847481,0.0,32,True
501,personnel infirmier,NOM NOM,54,614.338342,0.0,24,True
540,soins de longue durée,NOM PRP ADJ NOM,48,746.993638,0.0,44,True
609,conduite automobile,NOM ADJ,44,681.308777,0.0,16,True


### **Extraction de termes existant dans la taxonomie**

In [87]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,services,NOM,3252,-,-,622,False,False
1,hôpital,NOM,1724,-,-,262,False,False
2,médecin,NOM,1590,-,-,364,False,False
3,centre,NOM,1348,-,-,448,False,False
4,carte,NOM,902,-,-,212,False,False
...,...,...,...,...,...,...,...,...
1530,itss et contraception,NOM KON NOM,16,286.332206,0.0,16,False,False
1529,équité salariale,NOM ADJ,16,288.022399,0.0,4,True,False
1528,consommation d'alcool soutien,NOM PRP NOM NOM,16,197.078397,0.0,16,False,False
1527,crlb et ird,NOM KON ADJ,16,294.375033,0.0,16,False,False


In [88]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    labels = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

labels

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [89]:
for t in df['Terme'].tolist():
    if t in labels:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,services,NOM,3252,-,-,622,False,False
1,hôpital,NOM,1724,-,-,262,False,True
2,médecin,NOM,1590,-,-,364,False,True
3,centre,NOM,1348,-,-,448,False,False
4,carte,NOM,902,-,-,212,False,False
...,...,...,...,...,...,...,...,...
1530,itss et contraception,NOM KON NOM,16,286.332206,0.0,16,False,False
1529,équité salariale,NOM ADJ,16,288.022399,0.0,4,True,False
1528,consommation d'alcool soutien,NOM PRP NOM NOM,16,197.078397,0.0,16,False,False
1527,crlb et ird,NOM KON ADJ,16,294.375033,0.0,16,False,False


In [90]:
# Par exemple, on peut extraire les termes MeSH qui ne sont pas dans la taxonomie, mais qui se trouvent dans le corpus d'intérêt
df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()

C:\Users\p1115145\AppData\Local\Temp\ipykernel_16748\4006263597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()


,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
57,assurance maladie,NOM NOM,318,-inf,1.0,228,True,False
501,personnel infirmier,NOM NOM,54,614.338342,0.0,24,True,False
609,conduite automobile,NOM ADJ,44,681.308777,0.0,16,True,False
834,femmes enceintes,NOM VER:pper,34,653.098399,0.0,30,True,False
850,milieu de travail,NOM PRP NOM,32,272.174845,0.0,20,True,False
931,moteur de recherche,NOM PRP NOM,30,484.901826,0.0,30,True,False
921,mobilité réduite,NOM ADJ,30,539.526167,0.0,20,True,False
1010,orthèses plantaires,NOM ADJ,28,445.03335,0.0,10,True,False
969,aide alimentaire,NOM ADJ,28,320.322241,0.0,28,True,False
1117,travailleurs sociaux,NOM ADJ,24,348.370917,0.0,20,True,False


In [91]:
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df['Corpus'] = acteur
df = df[["Corpus", "Terme", "Structure syntaxique",	"Fréquence (TF)", "Fréquence documentaire (DF)", "LLR", "isMeSHTerm", "isTaxoTerm" ]]
df.to_csv(output_path)